## test random environment with openai gym

In [2]:
import gym
import random

In [6]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [12]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0

    while not done:
        env.render()
        action = random.choice([0,1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f'Episode: {episode} Score: {score}')

Episode: 1 Score: 27.0
Episode: 2 Score: 17.0
Episode: 3 Score: 21.0
Episode: 4 Score: 22.0
Episode: 5 Score: 26.0
Episode: 6 Score: 22.0
Episode: 7 Score: 20.0
Episode: 8 Score: 27.0
Episode: 9 Score: 27.0
Episode: 10 Score: 17.0


## create a deep learning model with keras

In [13]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [26]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [27]:
model = build_model(states, actions)

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


## build agent with keras-rl

In [19]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [29]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy,
                nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [30]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

/home/amoel/anaconda3/envs/reinfenv/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 50000 steps ...
Interval 1 (0 steps performed)


/home/amoel/anaconda3/envs/reinfenv/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


    1/10000 [..............................] - ETA: 39:39 - reward: 1.0000

/home/amoel/anaconda3/envs/reinfenv/lib/python3.9/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 82s 8ms/step - reward: 1.0000
113 episodes - episode_reward: 88.345 [9.000, 200.000] - loss: 3.114 - mae: 19.085 - mean_q: 38.587

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 81s 8ms/step - reward: 1.0000
53 episodes - episode_reward: 185.981 [138.000, 200.000] - loss: 4.722 - mae: 37.480 - mean_q: 75.604

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 81s 8ms/step - reward: 1.0000
53 episodes - episode_reward: 191.377 [154.000, 200.000] - loss: 5.201 - mae: 40.386 - mean_q: 81.322

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 86s 9ms/step - reward: 1.0000
50 episodes - episode_reward: 196.800 [146.000, 200.000] - loss: 6.325 - mae: 39.295 - mean_q: 78.929

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 88s 9ms/step - reward: 1.0000
done, took 417.814 seconds


In [31]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 200.000, steps: 200
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 200.000, steps: 200
Episode 9: reward: 200.000, steps: 200
Episode 10: reward: 200.000, steps: 200
Episode 11: reward: 200.000, steps: 200
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 200.000, steps: 200
Episode 16: reward: 200.000, steps: 200
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 200.000, steps: 200
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 200.000, steps: 200
Episode 22: reward: 200.000, steps: 200
Episode 23: reward: 200.000, steps: 200
Episode 24: reward: 200.000, steps: 200
Episode 25: reward: 

In [43]:
_ = dqn.test(env, nb_episodes=5, visualize=True, )

Testing for 5 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


In [44]:
env.close()

## reloading agent from memory

In [45]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [46]:
del model
del dqn
del env

In [47]:
# dqn is deleted
dqn.test(env, nb_episodes=5, visualize=True)

NameError: name 'dqn' is not defined

In [48]:
# reinstantiate the model
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

/home/amoel/anaconda3/envs/reinfenv/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [49]:
# reload the saved weights
dqn.load_weights('dqn_weights.h5f')

In [50]:
_ = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...


/home/amoel/anaconda3/envs/reinfenv/lib/python3.9/site-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 200.000, steps: 200
Episode 3: reward: 200.000, steps: 200
Episode 4: reward: 200.000, steps: 200
Episode 5: reward: 200.000, steps: 200


In [51]:
env.close()